In [ ]:
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")

name='TraceLocation'
prefix='DMS_'
csv_path = '/usr/local/airflow/plugins'+'/'

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 5, 10, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '5 * * * *',
          tags=[prefix+name, 'Daily', '60mins']
)

In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [ ]:
datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y%m%d")

In [ ]:
sql = \
f"""
DECLARE @from DATE = '{datenow_mns1}'
DECLARE @to DATE = '{datenow_mns1}'

SELECT
BranchID,
SlsperID,
Lat,
Lng,
CustID,
Type as Typ,
CheckType = 'IO',
NumberCICO,
UpdateTime
from dbo.AR_SalespersonLocationTrace 
where CAST (UpdateTime as DATE) >= @from
and CAST (UpdateTime as DATE) <= @to
and [Type] = 'IO'

UNION ALL

SELECT
BranchID,
SlsperID,
Lat,
Lng,
CustID,
Type as Typ,
CheckType = 'OO',
NumberCICO,
UpdateTime
from dbo.AR_SalespersonLocationTrace 
where CAST (UpdateTime as DATE) >= @from
and CAST (UpdateTime as DATE) <= @to
and [Type] = 'OO'

UNION ALL

SELECT
BranchID,
SlsperID,
Lat,
Lng,
CustID,
Type,
CheckType = 'MaxAction',
a.NumberCICO,
UpdateTime
from dbo.AR_SalespersonLocationTrace a
INNER JOIN
(
SELECT
NumberCICO,
max(UpdateTime) as MaxUpdateTime
from dbo.AR_SalespersonLocationTrace a
where CAST (UpdateTime as DATE) >= @from
and CAST (UpdateTime as DATE) <= @to
and [Type] not in ('OO','IO')
group by NumberCICO
) b
ON a.NumberCICO = b.NumberCICO
and a.UpdateTime = MaxUpdateTime
where CAST (UpdateTime as DATE) >= @from
and CAST (UpdateTime as DATE) <= @to
and [Type] not in ('OO','IO')
"""
TRACE = get_ms_df(sql)

try:
    assert TRACE.shape[0] > 0
    bq_values_insert(TRACE, "d_checkin", 2)
except AssertionError:
    print("There is no data")

In [ ]:
dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

insert = PythonOperator(task_id="insert", python_callable=insert, dag=dag)

dummy_start >> insert